In [42]:

from osdatahub import FeaturesAPI, Extent, NGD
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import mapclassify as mc
from shapely.geometry import Point
import os
import numpy as np

key = os.environ.get('OS_API_KEY')
bbox = (384427,	397567, 385272,398221)
extent = Extent.from_bbox(bbox, "EPSG:4326")
crs = "EPSG:4326"

In [14]:
# collection = "trn-ntwk-road"
# roadlines = NGD(key, collection)
# # make the query and contact the API
# features = roadlines.query(max_results=500, extent=extent)

product = "Zoomstack_DistrictBuildings"
features = FeaturesAPI(key, product, extent)

results_gdf = gpd.GeoDataFrame.from_features(features['features'], crs=crs)

TypeError: 'FeaturesAPI' object is not subscriptable

In [45]:
def OSparam_feature(u,v,rad,product,key,clip):
    extent = Extent.from_radius((u,v), rad, "EPSG:27700")
    features = FeaturesAPI(key, product, extent)
    results = features.query(limit=500)
    if len(results['features']) == 0:
        out = 0
    TA_gdf = gpd.GeoDataFrame.from_features(results['features'])

    if clip == True:
        patch = Point(u,v).buffer(rad)
        d = {'col1': ['name1'], 'geometry': [patch]}
        patch_df = gpd.GeoDataFrame(d, crs="EPSG:27700")
        TA_gdf = TA_gdf.clip(patch)
        # TA_gdf = TA_gdf[TA_gdf.geometry.intersects(Point(u,v).buffer(rad))]

    try:
        gd = (TA_gdf['Theme'] == 'Buildings') & (~np.isnan(TA_gdf['RelH2']))
        out = np.average(TA_gdf['RelH2'][gd], weights=TA_gdf['Shape_Area'][gd])
    except TypeError:
        out = 0
    return out, TA_gdf



In [46]:
X = 529955
Y = 179159

radius = 1000
clip = True

# extent = Extent.from_radius((X,Y), radius, "EPSG:27700")
# features = FeaturesAPI(key, "Zoomstack_DistrictBuildings", extent)
# results = features.query(limit=500)
# print(results)

area, TA = OSparam_feature(X, Y, radius, "topographic_area",key=key, clip=clip)

In [51]:
map = folium.Map(location=[51, 1], zoom_start=12)

# sim_geo = gpd.GeoSeries(TA["geometry"]).simplify(tolerance=0.001)
# geo_j = TA.to_json()

TA = TA.to_crs(epsg=4326)

for _, r in TA.iterrows():
    #without simplifying the representation of each borough, the map might not be displayed
    #sim_geo = gpd.GeoSeries(r['geometry'])
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['RelH2']).add_to(geo_j)
    geo_j.add_to(map)
map

ValueError: Cannot transform naive geometries.  Please set a crs on the object first.